In [ ]:
from collections import namedtuple
from glob import glob
from itertools import groupby
from operator import itemgetter
from pathlib import Path
import pickle

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel as LDA

from matplotlib import pyplot as plt

import numpy as np

from pandas import DataFrame, read_csv, concat

import pyLDAvis as ldavis
import pyLDAvis.gensim

from scipy.spatial.distance import cdist, pdist, squareform

from sklearn.model_selection import train_test_split

ldavis.enable_notebook()
%matplotlib notebook
%precision 4

out = Path('../output/')

Candidate = namedtuple('Candidate', ['iterations', 'num_topics'])

def get_i_t(filename):
    _, content, document_type = filename.split('-')
    i, t, _ = content.split('_')
    return int(i[1:]), int(t[1:]), document_type.split('.')[0]

get_texts = lambda df: df[target].str.split()
tobows = lambda df, d: get_texts(df).apply(d.doc2bow)

report_types = 'ISA', 'PFR', 'DPFR'

test_size = .2
min_occurances = 2

In [ ]:
documents_by_type = {
    t: read_csv(out / f'norm_{t}.csv').dropna()
    for t in report_types
}

target = 'GLOMUNSTEM'

with open('election_by_type.pkl', 'rb') as fd:
    election_by_type = pickle.load(fd)

with open('dict_train_test.pkl', 'rb') as fd:
    dictionary_by_type, train_by_type, test_by_type = pickle.load(fd)

model_by_type = dict()
for document_type in election_by_type:
    c = election_by_type[document_type]
    filename = f'narrow-i{c.iterations:03}_t{c.num_topics:03}_d-{document_type}'
    model_by_type[document_type] = LDA.load(str(out / f'{filename}.lda'))

In [ ]:
for document_type in report_types:
    with open(document_type + '_topics_vectors.csv', 'w') as fd:
        tump = documents_by_type[document_type]
        dump = dictionary_by_type[document_type]
        mump = model_by_type[document_type]

        for _, doc in tump.iterrows():
            gs = doc[target]
            bow = dump.doc2bow(gs.split())

            topics = mump.get_document_topics(bow, minimum_probability=-1)
            print(doc['Anomaly_ID'], *map(itemgetter(1), topics), sep=',', file=fd)